## Расчет байесовской вероятности на примере погоды (дополнение)

**Задача**

_Дано_: записи наблюдений температуры за несколько лет. 

_Опеределить_ вероятность отрицательной температуры для каждого времени года по Баесовской модели.

Загрузим данные по погоде за несколько лет

In [2]:
import pandas as pd

In [50]:
weather = pd.read_csv('./data/weather.csv')
weather['Day'] = pd.to_datetime(weather['Day'])
weather

,Day,t
0,2008-01-01,0
1,2008-01-02,-5
2,2008-01-03,-11
3,2008-01-04,-11
4,2008-01-05,-12
...,...,...
3280,2016-12-27,1
3281,2016-12-28,-3
3282,2016-12-29,0
3283,2016-12-30,3


Сопоставим месяцам время года, для чего напишем функцию, принимающую номер месяца и возвращающую название времени года:

In [30]:
def timeOfYear(month_n):
    k = ['winter', 'spring', 'summer', 'autumn']
    return k[(abs(month_n%12)) // 3]

    

In [38]:
for x in range(1,13):
    print(x, timeOfYear(x))

1 winter
2 winter
3 spring
4 spring
5 spring
6 summer
7 summer
8 summer
9 autumn
10 autumn
11 autumn
12 winter


Всего в году 4 времени года. Каждое время года занимает примерно 1/4 часть года. Примем это отношение за вероятность каждого времени года. Однако таблица температуры может содержать записи, чье распределение по временам года не будет равномерным, так как 
- протяженность записей не обязательна кратна годам, 
- в месяцах разное количество дней, а следовательно продолжительность разных времен года может несущественно различатся друг от друга.

Из-за этого, необходимо расчитать точные отношения (вероятности) каждого времени года в имеющейся таблице.

Воспользуемся функцией сопоставления месяца с временем года и назначим каждой записи ее время года:



In [61]:
weather['TimeOfYear'] = pd.Series(map(timeOfYear, weather.Day.dt.month))
weather

,Day,t,TimeOfYear
0,2008-01-01,0,winter
1,2008-01-02,-5,winter
2,2008-01-03,-11,winter
3,2008-01-04,-11,winter
4,2008-01-05,-12,winter
...,...,...,...
3280,2016-12-27,1,winter
3281,2016-12-28,-3,winter
3282,2016-12-29,0,winter
3283,2016-12-30,3,winter


Посчитаем вероятность каждого времени года (% от всех записей)

In [67]:
from collections import Counter
timeOfYearCount = dict(Counter(weather.TimeOfYear))
probOfTimeYear = {}
for x, y in timeOfYearCount.items():
    probOfTimeYear[x] = y / weather.Day.count()
probOfTimeYear

{'winter': 0.2471841704718417,
 'spring': 0.2517503805175038,
 'summer': 0.2517503805175038,
 'autumn': 0.2493150684931507}

Возьмем формулу полной вероятности для события A:
#### $P(A) = P(B_1)P(A|B_1) + P(B_2)P(B_2|A) + ... + P(B_n)P(A|B_n)$